In [2]:
# Import Pandas
import pandas as pd

# Load  Metadata
metadata=pd.read_excel("Recommend.xlsx")

# Print the first three rows
metadata.head(3)

Name of Person              Food Ordered  Rating  \
0  Sree Lokeshwari.K  Zaikedaar Paneer Biryani     5.0   
1  Sree Lokeshwari.K   Special Chicken Biryani     4.0   
2  Sree Lokeshwari.K              Kaju Biryani     4.0   

              Name of Resturant        Date of order  Unnamed: 5  
0               Behrouz Biryani  2019-06-07 00:00:00         NaN  
1                  Kinara Grand  2019-07-21 00:00:00         NaN  
2  Abhi Ruchi Family Restuarant  2019-08-16 00:00:00         NaN

In [3]:
metadata.shape

(3080, 6)

In [4]:
metadata.columns

Index(['Name of Person', 'Food Ordered', 'Rating', 'Name of Resturant',
       'Date of order', 'Unnamed: 5'],
      dtype='object')

In [9]:
metadata["Food Ordered"].isnull().sum()
metadata["Food Ordered"]=metadata["Food Ordered"].fillna(" ")
metadata=metadata.drop_duplicates("Food Ordered", keep="first")
#df.drop_duplicates('Zone',keep='last')
metadata1=metadata.drop(["Name of Person","Date of order"],axis=1)

C:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# Calculate mean of vote average column
C = metadata['Rating'].mean()
print(C)

3.9430332922318123


In [13]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['Rating'].quantile(0.90)
print(m)

5.0


In [14]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['Rating'] >= m]
q_movies.shape

(223, 6)

In [15]:
#Print plot overviews of the first 5 movies.
metadata['Rating'].head()

0    5.0
1    4.0
2    4.0
3    5.0
4    4.0
Name: Rating, dtype: float64

In [16]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['Food Ordered'] = metadata['Food Ordered'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['Food Ordered'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(814, 518)

In [17]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[800:501]

[]

In [18]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [19]:
cosine_sim.shape

(814, 814)

In [20]:
cosine_sim[1]

array([0.19032312, 1.        , 0.2390872 , 0.13522785, 0.        ,
       0.14521719, 0.        , 0.        , 0.        , 0.07210066,
       0.34364692, 0.12886201, 0.        , 0.24163266, 0.13522785,
       0.        , 0.14521719, 0.19879586, 0.        , 0.13220187,
       0.34564722, 0.09311349, 0.        , 0.33473234, 0.46985425,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.14178382, 0.        , 0.        , 0.        ,
       0.        , 0.60141532, 0.        , 0.        , 0.14296335,
       0.09045489, 0.1337483 , 0.        , 0.        , 0.08522754,
       0.08068857, 0.08340467, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.25379022, 0.1089524 , 0.15903386, 0.30762227,
       0.09913599, 0.        , 0.        , 0.1719249 , 0.3323135 ,
       0.36470099, 0.        , 0.        , 0.08224668, 0.10341764,
       0.10723204, 0.09730542, 0.08072768, 0.10215564, 0.     

In [22]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['Food Ordered']).drop_duplicates()
indices[:10]

Food Ordered
Zaikedaar Paneer Biryani            0
Special Chicken Biryani             1
Kaju Biryani                        2
Mutton Small Family Pack Biryani    3
Veg Family Pack Combo               4
Tandoori Chicken                    5
Butterscotch Cake                   6
Meal for 1 Non Veg                  7
Apollo Fish                         8
Lebanese Chicken Shawarma Roll      9
dtype: int64

In [26]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(Food_Ordered, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[Food_Ordered]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['Food Ordered'].iloc[movie_indices]

In [27]:
get_recommendations('Special Chicken Biryani')

106            special chicken Biryani
501           Special Chicken Biryani 
116             special paneer biryani
515    Special Chicken Tikka Biryani  
778             mutton special biryani
149    Arabian Special Chicken Biryani
563            Chicken Special Briyani
41                     Chicken Biryani
113                    chicken biryani
250                    chicken Biryani
Name: Food Ordered, dtype: object